### IMPORTS

In [4]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process

### DATA

In [2]:
wl_cams = pd.read_json('../data/WELAB/welab_cameras.json')
wl_lens = pd.read_json('../data/WELAB/welab_lenses.json')
wl_audio = pd.read_json('../data/WELAB/welab_audio.json')
wl_lights = pd.read_json('../data/WELAB/welab_lights.json')

In [3]:
rc_cams = pd.read_json('../data/RC SERVICE/rcservice_cameras.json')
rc_lenses = pd.read_json('../data/RC SERVICE/rcservice_lenses.json')

In [4]:
ov_cams = pd.read_json('../data/OVIDE/ovide_cameras.json')
ov_lights = pd.read_json('../data/OVIDE/ovide_lights.json')
ov_lenses = pd.read_json('../data/OVIDE/ovide_lenses.json')
ov_audio = pd.read_json('../data/OVIDE/ovide_audio.json')

In [6]:
rental_places = pd.read_json('../data/CLEAN/rental_places.json') 

### TRANSFORMATION

In [7]:
def transform_data(data_frame, rental_places_df):    #this is a function to transform all data in each dataframe at once.
    
    # first I want to convert all column names to lowercase
    data_frame.columns = data_frame.columns.str.lower()

    # and also I want to rename the price a day column.
    data_frame.rename(columns={'price a day': 'price_a_day'}, inplace=True)
    data_frame.rename(columns={'rental': 'rental_place_name'}, inplace=True)

    # I also would like to convert all values in the DataFrame to lowercase
    data_frame = data_frame.map(lambda x: x.lower() if isinstance(x, str) else x)

    # And I need to standardise the rental_place names and IDs
    rental_id_mapping = dict(zip(rental_places_df['rental_place_name'].str.lower(), rental_places_df['rental_place_id']))

    # Update the 'rental_place' column in the DataFrame with rental IDs
    data_frame['rental_place_id'] = data_frame['rental_place_name'].str.lower().map(rental_id_mapping)

    # and lastly drop the original 'rental_place'
    data_frame.drop(columns=['rental_place_name'], inplace=True) 
    
    return data_frame

In [8]:
wl_cams_2 = transform_data(wl_cams, rental_places)
wl_lens_2 = transform_data(wl_lens, rental_places)
wl_audio_2 = transform_data(wl_audio, rental_places)
wl_lights_2 = transform_data(wl_lights, rental_places)

In [9]:
rc_cams_2 = transform_data(rc_cams, rental_places)
rc_lenses_2 = transform_data(rc_lenses, rental_places)

In [10]:
ov_audio_2 = transform_data(ov_audio, rental_places)
ov_cams_2 = transform_data(ov_cams, rental_places)
ov_lenses_2 = transform_data(ov_lenses, rental_places)
ov_lights_2 = transform_data(ov_lights, rental_places)

In [11]:
ov_cams_2['name'] = ov_cams_2.name.str.replace('cámara', '').str.replace('mini','')

The next functions helps creating an id table that would identify the products.

In [12]:
def perform_fuzzy_matching(df_list):
    product_id = 1
    id_mapping = {}

    for idx, df in enumerate(df_list):
        df['product_id'] = df.index + 1  # Creating a product_id column starting from 1

        df['tokens'] = df['name'].str.split()
        df['matched_id'] = 0  # Create a new column for the matched product ID

        for _, row in df.iterrows():
            found = False
            token_str = ' '.join(row['tokens'])  # Convert list of tokens to string

            for key, value in id_mapping.items():
                # Compare tokenized product names for similarity
                score = fuzz.ratio(token_str, key)

                # If similarity score is above a threshold (e.g., 80), consider them as potential matches
                if score > 98:
                    df.at[_, 'matched_id'] = value
                    found = True
                    break

            if not found:
                id_mapping[token_str] = product_id
                df.at[_, 'matched_id'] = product_id
                product_id += 1

    return df_list


In [13]:
cams_list = [wl_cams_2, rc_cams_2, ov_cams_2]
lens_list = [wl_lens_2, rc_lenses_2, ov_lenses_2]
audio_list = [wl_audio_2, ov_audio_2]
lights_list = [wl_lights_2, ov_lights_2]

In [14]:
cams = perform_fuzzy_matching(cams_list)

In [15]:
wl_cams_3, rc_cams_3, ov_cams_3 = cams

In [16]:
ov_cams_3.head()

,category,type,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,cameras,cine digital,arri,arri alexa 35 4k,pedir presupuesto,https://www.ovide.com/alquiler-es/camara-arri-...,3,1,"[arri, alexa, 35, 4k]",85
1,cameras,cine digital,arri,arri alexa lf,pedir presupuesto,https://www.ovide.com/alquiler-es/camara-arri-...,3,2,"[arri, alexa, lf]",49
2,cameras,cine digital,arri,arri alexa,pedir presupuesto,https://www.ovide.com/alquiler-es/arri-alexa-m...,3,3,"[arri, alexa]",86
3,cameras,cine digital,arri,arri alexa lf,pedir presupuesto,https://www.ovide.com/alquiler-es/camara-arri-...,3,4,"[arri, alexa, lf]",49
4,cameras,cine digital,arri,arri alexa sxt plus 4:3,pedir presupuesto,https://www.ovide.com/alquiler-es/arri-alexa-s...,3,5,"[arri, alexa, sxt, plus, 4:3]",87


In [17]:
rc_cams_3.head()

,category,type,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,cameras,analogica,arri,packs arricam lt,pedir presupuesto,http://www.rcservice.es/es/camaras-35mm-16mm/2...,2,1,"[packs, arricam, lt]",62
1,cameras,analogica,movicam,packs movicam compact 35,pedir presupuesto,http://www.rcservice.es/es/camaras-35mm-16mm/2...,2,2,"[packs, movicam, compact, 35]",63
2,cameras,analogica,arri,packs arri 435,pedir presupuesto,http://www.rcservice.es/es/camaras-35mm-16mm/2...,2,3,"[packs, arri, 435]",64
3,cameras,analogica,arri,packs arri sr3,pedir presupuesto,http://www.rcservice.es/es/camaras-35mm-16mm/2...,2,4,"[packs, arri, sr3]",65
4,cameras,digital,arri,arri alexa 35,pedir presupuesto,http://www.rcservice.es/es/alquiler-camaras-pe...,2,5,"[arri, alexa, 35]",47


In [18]:
wl_cams_3.head()

,category,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,cameras,phantom,phantom flex 4k 128gb,pedir presupuesto,https://welabplus.com/shop8/camaras/phantom/ph...,1,1,"[phantom, flex, 4k, 128gb]",1
1,cameras,gopro,gopro hero 11 black edition,85,https://welabplus.com/shop8/camaras/gopro/gopr...,1,2,"[gopro, hero, 11, black, edition]",2
2,cameras,gopro,gopro max 360,70,https://welabplus.com/shop8/camaras/gopro/gopr...,1,3,"[gopro, max, 360]",3
3,cameras,gopro,gopro hero 10 black edition,75,https://welabplus.com/shop8/camaras/gopro/gopr...,1,4,"[gopro, hero, 10, black, edition]",4
4,cameras,gopro,gopro hero 9 black edition,69,https://welabplus.com/shop8/camaras/gopro/gopr...,1,5,"[gopro, hero, 9, black, edition]",5


In [19]:
lens = perform_fuzzy_matching(lens_list)
audio = perform_fuzzy_matching(audio_list)
lights = perform_fuzzy_matching(lights_list)

In [20]:
wl_lens_3, rc_lenses_3, ov_lenses_3 = lens
wl_audio_3, ov_audio_3 = audio
wl_lights_3, ov_lights_3 = lights

Once the tables have been transformed individually it is time to concat them to create a single df for each category.

In [21]:
camera_rental_df = pd.concat([wl_cams_3, rc_cams_3, ov_cams_3],ignore_index=True)
lens_rental_df = pd.concat([wl_lens_3, rc_lenses_3, ov_lenses_3], ignore_index=True)
audio_rental_df = pd.concat([wl_audio_3, ov_audio_3], ignore_index=True)
lights_rental_df = pd.concat([wl_lights_3, ov_lights_3], ignore_index=True)

In [22]:
camera_rental_df

,category,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id,type
0,cameras,phantom,phantom flex 4k 128gb,pedir presupuesto,https://welabplus.com/shop8/camaras/phantom/ph...,1,1,"[phantom, flex, 4k, 128gb]",1,NaN
1,cameras,gopro,gopro hero 11 black edition,85,https://welabplus.com/shop8/camaras/gopro/gopr...,1,2,"[gopro, hero, 11, black, edition]",2,NaN
2,cameras,gopro,gopro max 360,70,https://welabplus.com/shop8/camaras/gopro/gopr...,1,3,"[gopro, max, 360]",3,NaN
3,cameras,gopro,gopro hero 10 black edition,75,https://welabplus.com/shop8/camaras/gopro/gopr...,1,4,"[gopro, hero, 10, black, edition]",4,NaN
4,cameras,gopro,gopro hero 9 black edition,69,https://welabplus.com/shop8/camaras/gopro/gopr...,1,5,"[gopro, hero, 9, black, edition]",5,NaN
...,...,...,...,...,...,...,...,...,...,...
119,cameras,sony,sony pxw-fx6,pedir presupuesto,https://www.ovide.com/alquiler-es/camara-sony-...,3,30,"[sony, pxw-fx6]",23,video digital
120,cameras,sony,sony pxw-fx9,pedir presupuesto,https://www.ovide.com/alquiler-es/camara-sony-...,3,31,"[sony, pxw-fx9]",22,video digital
121,cameras,gopro,gopro hero10 black edition,pedir presupuesto,https://www.ovide.com/alquiler-es/mini-camara-...,3,32,"[gopro, hero10, black, edition]",103,video digital
122,cameras,gopro,gopro hero11 black edition,pedir presupuesto,https://www.ovide.com/alquiler-es/mini-camara-...,3,33,"[gopro, hero11, black, edition]",104,video digital


In [23]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(camera_rental_df):
    # Replace missing values with "digital" in column: 'type'
    camera_rental_df = camera_rental_df.fillna({'type': "digital"})
    camera_rental_df['type'] = camera_rental_df['type'].apply(lambda x: x if 'digital' not in x else 'digital')
    
    
    return camera_rental_df

camera_rental_df_clean = clean_data(camera_rental_df.copy())
camera_rental_df_clean.head()

,category,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id,type
0,cameras,phantom,phantom flex 4k 128gb,pedir presupuesto,https://welabplus.com/shop8/camaras/phantom/ph...,1,1,"[phantom, flex, 4k, 128gb]",1,digital
1,cameras,gopro,gopro hero 11 black edition,85,https://welabplus.com/shop8/camaras/gopro/gopr...,1,2,"[gopro, hero, 11, black, edition]",2,digital
2,cameras,gopro,gopro max 360,70,https://welabplus.com/shop8/camaras/gopro/gopr...,1,3,"[gopro, max, 360]",3,digital
3,cameras,gopro,gopro hero 10 black edition,75,https://welabplus.com/shop8/camaras/gopro/gopr...,1,4,"[gopro, hero, 10, black, edition]",4,digital
4,cameras,gopro,gopro hero 9 black edition,69,https://welabplus.com/shop8/camaras/gopro/gopr...,1,5,"[gopro, hero, 9, black, edition]",5,digital


In [31]:
camera_rental_df_clean.shape

(124, 10)

In [ ]:
#camera_rental_df_clean.to_json('../data/CLEAN/camera_df.json', orient="records")

In [24]:
lens_rental_df.head()

,category,brand,type,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,lenses,leitz,lpl-prime,"set lentes leitz elsie (18,25,35,50,75,100 mm)",850,https://welabplus.com/shop8/opticas/lentes-ful...,1,1,"[set, lentes, leitz, elsie, (18,25,35,50,75,10...",1
1,lenses,tokina,pl-prime,"set x5 lentes tokina vista 18,25,35,50,85 mm",225,https://welabplus.com/shop8/opticas/lentes-ful...,1,2,"[set, x5, lentes, tokina, vista, 18,25,35,50,8...",2
2,lenses,tokina,pl-prime,tokina vista cinema prime 18mm,50,https://welabplus.com/shop8/opticas/lentes-ful...,1,3,"[tokina, vista, cinema, prime, 18mm]",3
3,lenses,tokina,pl-prime,tokina vista cinema prime 25mm,50,https://welabplus.com/shop8/opticas/lentes-ful...,1,4,"[tokina, vista, cinema, prime, 25mm]",4
4,lenses,tokina,pl-prime,tokina vista cinema prime 35mm,50,https://welabplus.com/shop8/opticas/lentes-ful...,1,5,"[tokina, vista, cinema, prime, 35mm]",5


In [ ]:
#lens_rental_df.to_json('../data/CLEAN/lens_df.json', orient="records")

In [25]:
audio_rental_df.head()

,category,brand,type,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,audio,zoom,audio,zoom f6 multitrack,42,https://welabplus.com/shop8/audio/grabadoras/z...,1,1,"[zoom, f6, multitrack]",1
1,audio,zoom,audio,zoom h6,25,https://welabplus.com/shop8/audio/grabadoras/z...,1,2,"[zoom, h6]",2
2,audio,tascam,audio,tascam dr-05,10,https://welabplus.com/shop8/audio/grabadoras/t...,1,3,"[tascam, dr-05]",3
3,audio,zoom,audio,zoom h5,22,https://welabplus.com/shop8/audio/grabadoras/z...,1,4,"[zoom, h5]",4
4,audio,tascam,audio,tascam dr-40,20,https://welabplus.com/shop8/audio/grabadoras/t...,1,5,"[tascam, dr-40]",5


In [ ]:
#audio_rental_df.to_json('../data/CLEAN/audio_df.json', orient="records")

In [26]:
lights_rental_df.head()

,category,subcatgory,type,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id,subcategory
0,lighting,bastidores,telas-y-bastidores,avenger,kit x3 banderas hollywood avenger,12,https://welabplus.com/shop8/iluminacion/telas-...,1,1,"[kit, x3, banderas, hollywood, avenger]",1,NaN
1,lighting,bastidores,telas-y-bastidores,avenger,bandera hollywood avenger,6,https://welabplus.com/shop8/iluminacion/telas-...,1,2,"[bandera, hollywood, avenger]",2,NaN
2,lighting,bastidores,telas-y-bastidores,avenger,kit x2 bastidores abatibles 24×36 cm,12,https://welabplus.com/shop8/iluminacion/telas-...,1,3,"[kit, x2, bastidores, abatibles, 24×36, cm]",3,NaN
3,lighting,sujecion,accesorios-iluminacion,lastolite,fondo croma-key plegable verde y azul,20,https://welabplus.com/shop8/iluminacion/acceso...,1,4,"[fondo, croma-key, plegable, verde, y, azul]",4,NaN
4,lighting,sujecion,accesorios-iluminacion,unknown,estico 1×1,2,https://welabplus.com/shop8/iluminacion/acceso...,1,5,"[estico, 1×1]",5,NaN


In [27]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(lights_rental_df):
    lights_rental_df['subcatgory'] = lights_rental_df['subcatgory'].fillna(lights_rental_df['subcategory'])
    lights_rental_df = lights_rental_df.drop(columns=['subcategory'])
    return lights_rental_df

lights_rental_df_clean = clean_data(lights_rental_df.copy())
lights_rental_df_clean.head()

,category,subcatgory,type,brand,name,price_a_day,link,rental_place_id,product_id,tokens,matched_id
0,lighting,bastidores,telas-y-bastidores,avenger,kit x3 banderas hollywood avenger,12,https://welabplus.com/shop8/iluminacion/telas-...,1,1,"[kit, x3, banderas, hollywood, avenger]",1
1,lighting,bastidores,telas-y-bastidores,avenger,bandera hollywood avenger,6,https://welabplus.com/shop8/iluminacion/telas-...,1,2,"[bandera, hollywood, avenger]",2
2,lighting,bastidores,telas-y-bastidores,avenger,kit x2 bastidores abatibles 24×36 cm,12,https://welabplus.com/shop8/iluminacion/telas-...,1,3,"[kit, x2, bastidores, abatibles, 24×36, cm]",3
3,lighting,sujecion,accesorios-iluminacion,lastolite,fondo croma-key plegable verde y azul,20,https://welabplus.com/shop8/iluminacion/acceso...,1,4,"[fondo, croma-key, plegable, verde, y, azul]",4
4,lighting,sujecion,accesorios-iluminacion,unknown,estico 1×1,2,https://welabplus.com/shop8/iluminacion/acceso...,1,5,"[estico, 1×1]",5


In [ ]:
#lights_rental_df_clean.to_json('../data/CLEAN/lights_df.json', orient="records")

### MongoDB

In [39]:
#%pip install pymongo

In [1]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)


In [7]:
# I create the mongo database
db = client['AHAB_database']



In [5]:
cameras_data = pd.read_json('../data/CLEAN/camera_df.json').to_dict(orient='records')
lenses_data = pd.read_json('../data/CLEAN/lens_df.json').to_dict(orient='records')
lighting_data = pd.read_json('../data/CLEAN/lights_df.json').to_dict(orient='records')
audio_data = pd.read_json('../data/CLEAN/audio_df.json').to_dict(orient='records')
places = pd.read_json('../data/CLEAN/rental_places.json').to_dict(orient='records')

In [44]:
# And last but not least I upload data to MongoDB collections

db['cameras_collection'].insert_many(cameras_data)
db['lenses_collection'].insert_many(lenses_data)
db['lighting_collection'].insert_many(lighting_data)
db['audio_collection'].insert_many(audio_data)
db['rental_places'].insert_many(places)

InsertManyResult([ObjectId('6568c193642e60585b613652'), ObjectId('6568c193642e60585b613653'), ObjectId('6568c193642e60585b613654')], acknowledged=True)

In [8]:
collection = db['rental_places'] 

In [10]:
places

[{'rental_place_id': 1,
  'rental_place_name': 'WELAB',
  'address': 'C. de Albasanz, 59, 28037, Madrid',
  'latitude': 40.43856,
  'longitude': -3.62394,
  'email': 'info@welab.es',
  'phone': '0034 911823355',
  'website': 'https://www.welabplus.com'},
 {'rental_place_id': 2,
  'rental_place_name': 'RC SERVICE',
  'address': 'Avda. Prado del Espino, 1, 28660, Boadilla del Monte, Madrid',
  'latitude': 40.39606,
  'longitude': -3.85031,
  'email': 'info@rcservice.es',
  'phone': '0034 911855868',
  'website': 'https://www.rcservice.es'},
 {'rental_place_id': 3,
  'rental_place_name': 'OVIDE',
  'address': 'Av. Montes de Oca, 19, Nave 23, 28703, San Sebastián de los Reyes, Madrid',
  'latitude': 40.55172,
  'longitude': -3.61149,
  'email': 'madrid@ovide.com',
  'phone': '0034 915670407',
  'website': 'https://www.ovide.com/'}]

In [11]:
collection.delete_many({}) 

DeleteResult({'n': 3, 'ok': 1.0}, acknowledged=True)

In [12]:
collection.insert_many(places)

InsertManyResult([ObjectId('656be62e1735fe170b0687eb'), ObjectId('656be62e1735fe170b0687ec'), ObjectId('656be62e1735fe170b0687ed')], acknowledged=True)